In [1]:
import pandas as pd
import numpy as np
import torch
import os
# os.chdir('/home/jiageng/Documents/fhr/pipeline/')
os.chdir('/home/jiageng/Documents/fhr/pygcn/pygcn')
import snf

In [2]:
def stdNormalize(df):
    std = df.std().fillna(1)
    mean = df - df.mean()
    df_norm = mean / std
    return np.array(df_norm)

In [3]:
def rowNormalize(mx):
    """Row-normalize matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = np.diag(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [4]:
from scipy.sparse import coo_matrix

def sparse_tensor_from_arr(arr):
    """Convert numpy array to torch sparse tensor"""
    # Convert numpy array to scipy sparse matrix
    sparse_sp = coo_matrix(arr)
    
    # Convert scipy sparse matrix to torch sparse tensor
    sparse_tensor = torch.sparse_coo_tensor(
        torch.LongTensor([sparse_sp.row, sparse_sp.col]),
        torch.FloatTensor(sparse_sp.data),
        torch.Size(sparse_sp.shape)
    )
    
    return sparse_tensor

Prepare labels

In [5]:
fhr_data = pd.read_csv('/home/jiageng/Documents/fhr/annotations/fhr-annotations.tsv',sep='\t').set_index('PUBLIC_ID').query('risk != -1')
fhr_data['risk'] += 1

Prepare features

In [8]:
data = pd.read_csv('/home/jiageng/Documents/fhr/matrices/broad_cn_matrix.tsv',sep='\t').set_index('PUBLIC_ID')

In [11]:
data.describe()

,SeqWGS_Cp_10p14,SeqWGS_Cp_10q23,SeqWGS_Cp_11p15,SeqWGS_Cp_11q23,SeqWGS_Cp_12p13,SeqWGS_Cp_12q21,SeqWGS_Cp_13q14,SeqWGS_Cp_13q34,SeqWGS_Cp_14q23,SeqWGS_Cp_15q15,...,SeqWGS_Cp_5p15,SeqWGS_Cp_5q31,SeqWGS_Cp_6p22,SeqWGS_Cp_6q25,SeqWGS_Cp_7p14,SeqWGS_Cp_7q22,SeqWGS_Cp_8p22,SeqWGS_Cp_8q24,SeqWGS_Cp_9p13,SeqWGS_Cp_9q33
count,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,...,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000,924.000000
mean,0.015098,0.026733,0.227654,0.277808,-0.068431,-0.013188,-0.401000,-0.341593,-0.158784,0.369517,...,0.273979,0.261853,0.140313,-0.092735,0.199197,0.216338,-0.149380,0.044667,0.340076,0.364253
std,0.176041,0.148618,0.316424,0.334422,0.275947,0.194094,0.444205,0.436672,0.376785,0.377280,...,0.306626,0.311341,0.254870,0.386022,0.279982,0.275104,0.347613,0.299093,0.347467,0.326237
min,-1.381000,-0.954000,-0.978000,-0.934000,-1.271000,-1.084000,-1.941000,-1.036000,-0.994000,-0.286000,...,-0.938000,-0.849000,-0.965000,-1.005000,-0.976000,-0.241000,-1.200000,-0.974000,-0.964000,-0.293000
25%,0.006000,0.006000,0.014000,0.016000,-0.001250,0.005000,-0.911000,-0.882250,-0.087250,0.021000,...,0.012000,0.014000,0.012000,-0.030250,0.013000,0.014000,-0.131250,0.008000,0.017000,0.020000
50%,0.015000,0.013000,0.027000,0.038000,0.012000,0.014000,-0.154000,-0.017500,0.009000,0.327000,...,0.035000,0.032500,0.021000,0.011000,0.024000,0.026000,0.008000,0.015500,0.401500,0.461500
75%,0.021000,0.021000,0.549000,0.573250,0.021000,0.022000,0.009000,0.009000,0.019000,0.594000,...,0.567250,0.567000,0.107500,0.023000,0.534000,0.545000,0.016000,0.026250,0.583000,0.584000
max,1.007000,1.019000,1.424000,1.764000,1.022000,1.022000,0.761000,0.774000,1.025000,1.602000,...,1.143000,1.052000,1.126000,1.025000,1.530000,1.530000,1.009000,3.053000,1.729000,1.574000


In [107]:
feature_norm_mtd = '' 

if feature_norm_mtd == 'stdnorm':
    features = torch.tensor(stdNormalize(data),dtype=torch.float32)
elif feature_norm_mtd == 'rownorm':
    features = torch.tensor(rowNormalize(data.values),dtype=torch.float32)
else:
    features = torch.tensor(data.values,dtype=torch.float32)
    
print(features.size())

torch.Size([924, 42])


In [25]:
risk = fhr_data.reindex(pd.Index(data.index)).loc[data.index]['risk'].fillna(-1).astype(int)
labels = torch.tensor(risk.values,dtype=torch.long)
print(len(labels))

924


In [26]:
# missing values are set to -1
risk.value_counts()

risk
 1    513
 2    259
 3     92
-1     60
Name: count, dtype: int64

Assign training and validation samples

In [116]:
idx_labeled = np.where(risk != -1)[0]
idx_unlabeled = np.where(risk == -1)[0]
idx_val = torch.tensor(idx_labeled[1::3]) # k-fold split
idx_test = idx_val
idx_train = torch.tensor([idx for idx in idx_labeled if idx not in idx_val])
print('Un-labeled',len(idx_unlabeled))
print('Labeled train',len(idx_train))
print('Labeled val',len(idx_val))

Un-labeled 60
Labeled train 576
Labeled val 288


Prepare adjacency matrix

In [117]:
aff = snf.make_affinity(stdNormalize(data), metric='euclidean', K=20, mu=0.5)
print(aff.shape)

/home/jiageng/Documents/fhr/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jiageng/Documents/fhr/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(924, 924)


In [126]:
pctile = 0.97
threshold = np.quantile(aff,pctile)
print(pctile, threshold)

0.97 0.013048945651615969


In [148]:
aff_clip = aff.copy()
aff_clip[aff_clip < threshold] = 0
adj = sparse_tensor_from_arr(rowNormalize(aff_clip))

# Train Model

In [168]:
import time
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args(['--epochs=200','--dropout=0.5','--hidden=16','--weight_decay=5e-4','--seed=2349039492328'])
args.cuda = not args.no_cuda and torch.cuda.is_available()
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.4009 acc_train: 0.0000 loss_val: 1.3173 acc_val: 0.4792 time: 0.4861s
Epoch: 0002 loss_train: 1.3462 acc_train: 0.4132 loss_val: 1.2691 acc_val: 0.6042 time: 0.7230s
Epoch: 0003 loss_train: 1.3069 acc_train: 0.5260 loss_val: 1.2258 acc_val: 0.6250 time: 0.4668s
Epoch: 0004 loss_train: 1.2666 acc_train: 0.5469 loss_val: 1.1861 acc_val: 0.6424 time: 0.4957s
Epoch: 0005 loss_train: 1.2358 acc_train: 0.5660 loss_val: 1.1497 acc_val: 0.6424 time: 0.4776s
Epoch: 0006 loss_train: 1.2026 acc_train: 0.5694 loss_val: 1.1144 acc_val: 0.6424 time: 0.4641s
Epoch: 0007 loss_train: 1.1771 acc_train: 0.5694 loss_val: 1.0807 acc_val: 0.6424 time: 0.5088s
Epoch: 0008 loss_train: 1.1460 acc_train: 0.5694 loss_val: 1.0490 acc_val: 0.6424 time: 0.4883s
Epoch: 0009 loss_train: 1.1227 acc_train: 0.5694 loss_val: 1.0202 acc_val: 0.6424 time: 0.4775s
Epoch: 0010 loss_train: 1.1017 acc_train: 0.5694 loss_val: 0.9947 acc_val: 0.6424 time: 0.5486s
Epoch: 0011 loss_train: 1.0761 acc_train

In [154]:
# FFNN control - only consider self values
adj = torch.eye(adj.size(0)).to_sparse()

In [167]:
# Use original affinity network. Do not clip small values
# slow and not good
adj = sparse_tensor_from_arr(rowNormalize(aff))

In [164]:
adj = sparse_tensor_from_arr(rowNormalize(aff_clip))

Findings

When dropout is 0, using adj matrix is better than identity matrix

But if dropout is 0.5 or 0.9, using identity matrix is better than adj matrix